In [1]:
%%capture
%pip install gurobipy
import warnings
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import Model
from gurobipy import GRB
warnings.filterwarnings('ignore')

In [2]:
# Question 2

# Given data
N = 4
d = [30, 20, 40, 10]  # Demand per month
S = 500  # Setup cost
c = 100  # Production cost per unit
h = 2  # Holding cost per unit per month
M = 100  # A large number

# Initialize model
model = Model("Production Planning")

# Decision variables
x = model.addVars(N, vtype=GRB.CONTINUOUS, name="x")  # Production
I = model.addVars(N, vtype=GRB.CONTINUOUS, name="I")  # Inventory
y = model.addVars(N, vtype=GRB.BINARY, name="y")  # Setup indicator

# Objective function: Minimize total cost
model.setObjective(
    sum(S * y[i] + c * x[i] + h * I[i] for i in range(N)),
    GRB.MINIMIZE
)

# Constraints
# Inventory balance constraints
model.addConstr(I[0] == x[0] - d[0], "Inventory_0")
for i in range(1, N):
    model.addConstr(I[i] == I[i-1] + x[i] - d[i], f"Inventory_{i}")

# No inventory left at the end
model.addConstr(I[N-1] == 0, "Final_Inventory")

# Setup constraint
for i in range(N):
    model.addConstr(x[i] <= M * y[i], f"Setup_{i}")

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    total_cost = model.objVal
    print(f"Optimal Total Cost: ${total_cost}")
    for i in range(N):
        print(f"Month {i+1}: Produce {x[i].x:.0f} units, Inventory {I[i].x:.0f} units, Setup {int(y[i].x)}")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 12 columns and 20 nonzeros
Model fingerprint: 0x96befdde
Variable types: 8 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 4e+01]
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 12 nonzeros
Variable types: 5 continuous, 3 integer (3 binary)
Found heuristic solution: objective 12000.000000

Root relaxation: objective 1.076000e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

In [3]:
# Question 3

# Given data
T = 6
d = [40, 30, 50, 20, 60, 30]  # Demand per month
C = [50, 40, 60, 40, 70, 50]  # Production capacity per month
c = [100, 110, 105, 115, 95, 100]  # Production cost per unit per month
S = 500  # Setup cost per month
h = 5  # Holding cost per unit per month
I0 = 10  # Initial inventory

# Initialize model
model = Model("Production Planning")

# Decision variables
x = model.addVars(T, vtype=GRB.CONTINUOUS, name="x")  # Production
I = model.addVars(T, vtype=GRB.CONTINUOUS, name="I")  # Inventory
y = model.addVars(T, vtype=GRB.BINARY, name="y")  # Setup indicator

# Objective function: Minimize total cost
model.setObjective(
    sum(S * y[t] + c[t] * x[t] + h * I[t] for t in range(T)),
    GRB.MINIMIZE
)

# Constraints
# Inventory balance constraints
model.addConstr(I[0] == I0 + x[0] - d[0], "Inventory_0")
for t in range(1, T):
    model.addConstr(I[t] == I[t-1] + x[t] - d[t], f"Inventory_{t}")

# Production capacity constraints
for t in range(T):
    model.addConstr(x[t] <= C[t], f"Capacity_{t}")

# Setup constraint
for t in range(T):
    model.addConstr(x[t] <= C[t] * y[t], f"Setup_{t}")

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    total_cost = model.objVal
    print(f"Optimal Total Cost: ${total_cost}")
    for t in range(T):
        print(f"Month {t+1}: Produce {x[t].x:.0f} units, Inventory {I[t].x:.0f} units, Setup {int(y[t].x)}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 18 rows, 18 columns and 35 nonzeros
Model fingerprint: 0x09b4ae27
Variable types: 12 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [5e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 7e+01]
Presolve removed 14 rows and 11 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 10 nonzeros
Variable types: 4 continuous, 3 integer (3 binary)
Found heuristic solution: objective 25400.000000

Root relaxation: objective 2.478333e+04, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 24783.3333

In [4]:
# Question 4

# Define customer types
customer_types = ["Kids", "Drivers", "Workers"]
num_customers = {"Kids": 300, "Drivers": 240, "Workers": 600}

# Maximum price each group is willing to pay
willingness_to_pay = {
    "Kids": [2.69, 1.39, 1.09, 4.29],
    "Drivers": [2.99, 0.99, 1.29, 4.89],
    "Workers": [2.59, 0.99, 1.19, 4.19]
}

# Create a Gurobi model
m = Model("BigBurgerPricing")

# Decision variables: Prices
p_B = m.addVar(lb=0, name="Burger_Price")
p_F = m.addVar(lb=0, name="Fries_Price")
p_S = m.addVar(lb=0, name="SoftDrink_Price")
p_C = m.addVar(lb=0, name="Combo_Price")

# Decision variables: Quantity purchased by each customer type
x_B = {c: m.addVar(lb=0, ub=num_customers[c], vtype=GRB.CONTINUOUS, name=f"x_B_{c}") for c in customer_types}
x_F = {c: m.addVar(lb=0, ub=num_customers[c], vtype=GRB.CONTINUOUS, name=f"x_F_{c}") for c in customer_types}
x_S = {c: m.addVar(lb=0, ub=num_customers[c], vtype=GRB.CONTINUOUS, name=f"x_S_{c}") for c in customer_types}
x_C = {c: m.addVar(lb=0, ub=num_customers[c], vtype=GRB.CONTINUOUS, name=f"x_C_{c}") for c in customer_types}

# Constraints: Price must not exceed willingness to pay
for c in customer_types:
    m.addConstr(p_B <= willingness_to_pay[c][0])
    m.addConstr(p_F <= willingness_to_pay[c][1])
    m.addConstr(p_S <= willingness_to_pay[c][2])
    m.addConstr(p_C <= willingness_to_pay[c][3])

# Constraint: Combo price should be at most sum of individual prices
m.addConstr(p_C <= p_B + p_F + p_S)

# Constraint: Customers only buy one item per visit
for c in customer_types:
    m.addConstr(x_B[c] + x_F[c] + x_S[c] + x_C[c] == num_customers[c])

# Objective Function: Maximize revenue
m.setObjective(
    sum(x_B[c] * p_B + x_F[c] * p_F + x_S[c] * p_S + x_C[c] * p_C for c in customer_types),
    GRB.MAXIMIZE
)

# Solve the model
m.optimize()

# Print optimal prices
if m.status == GRB.OPTIMAL:
    print(f"Optimal Burger Price: ${p_B.X:.2f}")
    print(f"Optimal Fries Price: ${p_F.X:.2f}")
    print(f"Optimal Soft Drink Price: ${p_S.X:.2f}")
    print(f"Optimal Combo Price: ${p_C.X:.2f}")
    print(f"Maximal Revenue: ${m.objVal:.2f}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 16 columns and 28 nonzeros
Model fingerprint: 0x82e279a7
Model has 12 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e+02, 6e+02]
  RHS range        [1e+00, 6e+02]
Presolve removed 12 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 3 columns
Presolve time: 0.00s
Presolved: 10 rows, 18 columns, 28 nonzeros
Presolved model has 3 bilinear constraint(s)
Found heuristic solution: objective 2952.6000000
Variable types: 18 continuous, 0 integer (0 binary)
Found heuristic solution: objective 4776.6000000

Root 

In [5]:
# Question 5
from gurobipy import Model, GRB, quicksum
import numpy as np

# Data
customers = [1, 2, 3, 4, 5]
depot = 0
demands = {1: 20, 2: 15, 3: 30, 4: 25, 5: 10}
vehicle_capacity = 150
num_vehicles = 3
distance_matrix = np.array([
    [0, 10, 20, 15, 25, 30],
    [10, 0, 18, 14, 21, 24],
    [20, 18, 0, 12, 30, 27],
    [15, 14, 12, 0, 15, 20],
    [25, 21, 30, 15, 0, 18],
    [30, 24, 27, 20, 18, 0]
])

# Model
model = Model("VRP")

# Decision Variables
x = model.addVars(num_vehicles, range(6), range(6), vtype=GRB.BINARY, name="x")
u = model.addVars(range(6), vtype=GRB.CONTINUOUS, name="u")

# Objective: Minimize total distance
model.setObjective(
    quicksum(distance_matrix[i, j] * x[k, i, j] for k in range(num_vehicles) for i in range(6) for j in range(6)),
    GRB.MINIMIZE
)

# Constraints
# Each customer is visited exactly once
for i in customers:
    model.addConstr(quicksum(x[k, i, j] for k in range(num_vehicles) for j in range(6)) == 1)

# Vehicles start and end at the depot
for k in range(num_vehicles):
    model.addConstr(quicksum(x[k, depot, j] for j in range(1, 6)) == 1)
    model.addConstr(quicksum(x[k, j, depot] for j in range(1, 6)) == 1)

# Capacity constraints
for k in range(num_vehicles):
    model.addConstr(quicksum(demands[i] * quicksum(x[k, i, j] for j in range(6)) for i in customers) <= vehicle_capacity)

# Subtour elimination
for k in range(num_vehicles):
    for i in customers:
        for j in customers:
            if i != j:
                model.addConstr(u[i] - u[j] + vehicle_capacity * x[k, i, j] <= vehicle_capacity - demands[j])

# Solve model
model.optimize()

# Extract solution
if model.status == GRB.OPTIMAL:
    for k in range(num_vehicles):
        print(f"Vehicle {k+1} route:")
        for i in range(6):
            for j in range(6):
                if x[k, i, j].x > 0.5:
                    print(f"{i} -> {j}")
    print(f"Total minimal distance: {model.objVal}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 74 rows, 114 columns and 390 nonzeros
Model fingerprint: 0xc5d6664a
Variable types: 6 continuous, 108 integer (108 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 75.0000000
Presolve removed 54 rows and 109 columns
Presolve time: 0.00s
Presolved: 20 rows, 5 columns, 40 nonzeros
Variable types: 5 continuous, 0 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 75 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.500000000000e+01, best bound 7.500000000000e+01, g